# StorageSC: Smart Connector for Storing and Serving Humidity and Temperature Measurements

This notebook is a knowledge base and smart connector for storing humidity and temperature measurements in a remote GraphDB triples database. GraphDB is a highly-efficient and scalable RDF database that allows you to link diverse data, index it for semantic search, and enrich it via text analysis to build big knowledge graphs.

StorageSC has two react knowledge interactions that are triggered when the graph pattern of the post knowledge interactions from the smart connector knowledge base gets matched by the knowledge engine with the graph pattern of these react knowledge interactions. These interactions handle the bindings sent by the post side by matching them with the graph pattern and storing that in the GraphDB.

StorageSC also has two answer knowledge interactions for querying the GraphDB for temperature and humidity measurements made in a specified datetime range. These interactions are triggered by ask knowledge interactions coming from another notebook. The ask side sends bindings specifying the datetime range, which are then used by the answer side to query the GraphDB and return the requested measurements.

In summary, this notebook provides a smart connector for storing and serving humidity and temperature measurements in a remote GraphDB triples database. It receives live measurements from another smart connector notebook, stores them in the GraphDB, and allows querying for measurements made in a specified datetime range.

In [1]:
# Storage SC

In [2]:
import logging

import helpers as sp
from utils import *

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("StorageSC")

In [4]:
# Constants
# GraphDB endpoints
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
# prefixes of ontologies used in the graph patterns
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "ex": "http://example.org/",
}
# temp measurement graph pattern
TEMP_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""

# graph pattern describing a timeseries of temperature measurements with timestamp between ?startTimestamp and ?endTimestamp
TEMP_TIMESERIES_GRAPH_PATTERN = """?timeseries rdf:type ex:Timeseries .
                            ?timeseries ex:hasMeasurement ?meas .
                            ?timeseries ex:measuredAfter ?startTimestamp .
                            ?timeseries ex:measuredBefore ?endTimestamp .
                            ?meas rdf:type saref:Measurement .
                            ?meas saref:isMeasuredIn saref:TemperatureUnit .
                            ?meas saref:hasValue ?temp .
                            ?meas saref:hasTimestamp ?timestamp .
                            """

# humidity measurement graph pattern
HUMIDITY_MEAS_GRAPH_PATTERN = "EDIT THIS"

# graph pattern describing a timeseries of humidity measurements with timestamp between ?startTimestamp and ?endTimestamp
HUMIDITY_TIMESERIES_GRAPH_PATTERN = "EDIT THIS"

In [5]:
# parsing results of the querry to match the ask binding
def parse_response(response, start_timestamp, end_timestamp, timeseries):
    variables = response["head"]["vars"]
    bindings = response["results"]["bindings"]
    result = []
    for binding in bindings:
        measurement = {}
        for var in variables:
            # we add double quotes "<var>" to each variable (necessary to be compatible with the Knowledge Engine.
            measurement[var] = f'"{binding[var]["value"]}"'
        # Here we add timeseries, startTimestamp, endTimestamp to each measurement so that it matches the ask binding.
        measurement["timeseries"] = timeseries
        measurement["startTimestamp"] = start_timestamp
        measurement["endTimestamp"] = end_timestamp

        result.append(measurement)
    return result

In [6]:
# Return temp measurements in datetime range from GraphDB
def handle_temp_answer_measurements(query_bindings):
    for binding in query_bindings:
        logger.info(f"Handling answer for temp: {binding['timeseries']}")

        start_timestamp = binding["startTimestamp"]
        end_timestamp = binding["endTimestamp"]
        timeseries = binding["timeseries"]
        # This sparqle querry returns all measurements with timestamp between start_timestamp and end_timestamp
        sparql_query = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                            PREFIX saref: <https://w3id.org/saref#>
                            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

                            SELECT *

                            WHERE {{
                                ?meas rdf:type saref:Measurement .
                                ?meas saref:hasValue ?temp .
                                ?meas saref:isMeasuredIn saref:TemperatureUnit .
                                ?meas saref:hasTimestamp ?timestamp .
                                FILTER (xsd:dateTime(?timestamp) >= {start_timestamp}^^xsd:dateTime && xsd:dateTime(?timestamp) <= {end_timestamp}^^xsd:dateTime)
                                }}"""
        # running the sparql querry against GraphDB
        res = sp.run_sparql_query(READ_URL, sparql_query)

        # parsing results of the querry to match the ask binding
        result = parse_response(res, start_timestamp, end_timestamp, timeseries)

    return result


def handle_temp_react_measurements(bindings):
    # storing the live temp measurements in GraphDB
    sp.store_data_in_graphdb(
        graph_pattern=TEMP_MEAS_GRAPH_PATTERN,
        binding_set=bindings,
        prefixes=PREFIXES,
        read_url=READ_URL,
        write_url=WRITE_URL,
    )
    for binding in bindings:
        logger.info(f"Saving temperature measurement {binding['meas']}")
    return []

In [7]:
# Return humidity measurements in datetime range from GraphDB
def handle_humidity_answer_measurements(query_bindings):
    for binding in query_bindings:
        logger.info(f"Handling answer for humidity: {binding['timeseries']}")

        start_timestamp = binding["startTimestamp"]
        end_timestamp = binding["endTimestamp"]
        timeseries = binding["timeseries"]
        # This sparqle querry returns all measurements with timestamp between start_timestamp and end_timestamp
        sparql_query = f"EDIT THIS"
        # running the sparql querry against GraphDB
        res = sp.run_sparql_query(READ_URL, sparql_query)

        # parsing results of the querry to match the ask binding
        result = parse_response(res, start_timestamp, end_timestamp, timeseries)

    return result


def handle_humidity_react_measurements(bindings):
    # storing the live humidity measurements in GraphDB
    # ADD SOEMTHING HERE
    for binding in bindings:
        logger.info(f"Saving humidity measurement {binding['meas']}")
    return []

In [8]:
def start_storage_kb(kb_id, kb_name, kb_description, ke_endpoint):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    # register temp answer ki
    answer_temp_measurements_ki = register_answer_knowledge_interaction(
        TEMP_TIMESERIES_GRAPH_PATTERN,
        "answer-historical-temp-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    # register temp react ki
    react_temp_measurements_ki = register_react_knowledge_interaction(
        TEMP_MEAS_GRAPH_PATTERN,
        None,
        "react-temp-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    # register humidity answer ki
    answer_humidity_measurements_ki = "EDIT THIS"

    # register humidity react ki
    react_humidity_measurements_ki = "EDIT THIS"

    # start handle loop for handling answer and react kis
    start_handle_loop(
        {
            answer_temp_measurements_ki: handle_temp_answer_measurements,
            react_temp_measurements_ki: handle_temp_react_measurements,
            # ADD SOMETHING HERE
        },
        kb_id,
        ke_endpoint,
    )

In [9]:
start_storage_kb(
    "http://example.org/storage",
    "Storage",
    "GraphDB smart connector",
    "http://knowledge_engine:8280/rest/",
)

2023-06-26 00:01:13 INFO {"messageType":"error","message":"Deletion of knowledge base failed, because it could not be found."}
2023-06-26 00:01:13 INFO registered Storage
2023-06-26 00:01:13 INFO received issued knowledge interaction id: http://example.org/storage/interaction/answer-historical-temp-measurements
2023-06-26 00:01:13 INFO received issued knowledge interaction id: http://example.org/storage/interaction/react-temp-measurements
2023-06-26 00:01:21 INFO Saving temperature measurement <http://0.0.0.0:8001/thermostat/measurements/c7460f23-650b-4462-8320-f3372c5bc0e4>
2023-06-26 00:01:27 INFO Saving temperature measurement <http://0.0.0.0:8001/thermostat/measurements/30f1dc64-d554-44b1-89e0-aa0f6af0ec3f>
2023-06-26 00:01:34 INFO Saving temperature measurement <http://0.0.0.0:8001/thermostat/measurements/42bbf740-dddc-4158-ac1c-463bac003601>
2023-06-26 00:01:40 INFO Saving temperature measurement <http://0.0.0.0:8001/thermostat/measurements/73cb0a0b-7157-42e8-8b7f-1f9c938dcecc>
2

KeyboardInterrupt: 